In [3]:
#installera pandas och matplotlib, 
# och numpy om den inte kommer automatiskt med pandas
import json
import glob
import os
import re
import pandas as pd
import numpy as np
import time
import pprint
import matplotlib as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#Printar lite extra grejs som kan vara intressant om saker går fel
DEBUG = False

In [ ]:
#Genererar ett filträd. Måste ligga i samma mapp som datamapparna
# tree[date][industry][company][handle] är en lista med alla filer för den användaren (alltså: mentions, till, från).  

#TESTAD 19/3, Patrik har kontrollerat att längden på fillistan stämmer överrens med antal jsonfiler och att filnamn
# inte förekommer dubbelt. Koden är testad när notebooken ligger i samma mapp som två datamappar (2020-03-18, 2020-03-19).


tree = {}
INDUSTRIES = ["Airlines"]

#Lista med alla mappar i working directory som heter något i stil med 1111-11-11
collections = [d for d in glob.glob("*") if re.fullmatch(r'[0-9]{4}-[0-9]{2}-[0-9]{2}', d)]

for col in collections:
    tree[col] = {}
    for ind in INDUSTRIES:
        tree[col][ind] = {}
        
        #Den här fångar alla undermappar i /2020-xx-xx/<industry>/
        companies = [os.path.basename(x) for x in glob.glob(os.path.join(col, ind, "*"))]
        
        for comp in companies:
            tree[col][ind][comp] = {}
            
            #Traskar igenom "underfilträdet" för varje företag, alltså med /2020-xx-xx/industry/company/ som root. 
            #Lite snårig men finns bra dokumentation online.
            for root, dirs, files in os.walk(os.path.join(col, ind, comp), topdown=True):
                if dirs:
                    for handle in dirs:
                        tree[col][ind][comp][handle] = []
                else:
                    for f in files:
                        if f.find("tweet") == 0:
                            tree[col][ind][comp][os.path.basename(root)].append(os.path.join(root, f))
                            
#Printar en översikt av hela filträdet
if DEBUG:
    for c in tree.keys():
        print(c)
        for ind in tree[c].keys():
            print('\t', ind)
            for comp in tree[c][ind].keys():
                print('\t\t', comp)
                for handle in tree[c][ind][comp].keys():
                    print('\t\t\t', handle)
                    print('\t\t\t', "antal json:", str(len(tree[c][ind][comp][handle])))
                    for f in tree[c][ind][comp][handle]:
                        print('\t\t\t\t', f)

In [ ]:
#Läser in allt i flera DataFrames.
#ITEMS är de fält som tas med. Filtrerar direkt för att spara minne.
#Lite enkla utskrifter för att testa vad som tar tid.
ITEMS = ["created_at", "full_text", "id", "id_str", "metadata", "in_reply_to_status_id", "user"]

li = []
t_start = time.time()
for day in tree.keys():
    for company in tree[day]["Airlines"].keys():
        for path in list(tree[day]["Airlines"][company].values())[0]:
            df = pd.read_json(path)
            #Ta bort alla fält som inte finns i items
            df = df.filter(items=ITEMS)
            #Skapa en bool-series baserat på språket och behåll endast kolumner där språket känns igen som engelska.
            en = df.metadata.apply(lambda x: x["iso_language_code"] == "en")
            df = df[en]
            #Lägg till en kolumn som anger vilket företag tweeten insamlats från (alltså EJ avsändaren)
            df["associated_company"]=company
            li.append(df)
            
        
    
print("Time - read files:", time.time()-t_start, "seconds")
data = pd.concat(li)
print("Time - concat:", time.time()-t_start, "seconds")
print("Tweets:", data.shape[0])
print("Memory MB (with duplicates):", data.memory_usage(index=True, deep=True).sum() / 1000000)
data = data.drop_duplicates(subset="id")
print("Tweets (no duplicates):", data.shape[0])
print("Time - drop duplicates:", time.time()-t_start, "seconds")
print("Memory MB (no duplicates):", data.memory_usage(index=True, deep=True).sum() / 1000000)

data = data.set_index("id")
data['created_at'] = pd.to_datetime(data['created_at'])

In [ ]:
# Save to csv file
data.to_csv('data.csv')

In [6]:
# Read csv file
data = pd.read_csv("data.csv", index_col="id")
data['created_at'] = pd.to_datetime(data['created_at'])
data.head()

,created_at,full_text,id_str,metadata,in_reply_to_status_id,user,associated_company,sentiment
id,,,,,,,,
1240387060902526976,2020-03-18,"I agree with what you say Gaz, just like they ...",1240387060902526976,"{'iso_language_code': 'en', 'result_type': 're...",NaN,"{'id': 761670584350769153, 'id_str': '76167058...",Flybe,0.6239
1240355853531779072,2020-03-18,RT @ChaplainWEdin: Sad to hear of the situatio...,1240355853531779072,"{'iso_language_code': 'en', 'result_type': 're...",NaN,"{'id': 1009255824, 'id_str': '1009255824', 'na...",Flybe,-0.4767
1240351351479115777,2020-03-18,Has anyone had a refund back yet from @flybe ?...,1240351351479115776,"{'iso_language_code': 'en', 'result_type': 're...",NaN,"{'id': 861615007, 'id_str': '861615007', 'name...",Flybe,0.4122
1240311020028973056,2020-03-18,https://t.co/WWfcHihpSc\n\nLets not forget our...,1240311020028973056,"{'iso_language_code': 'en', 'result_type': 're...",NaN,"{'id': 2330120119, 'id_str': '2330120119', 'na...",Flybe,-0.1615
1240307787080450048,2020-03-18,RT @JasonUnsworthAA: While airlines are at ris...,1240307787080450048,"{'iso_language_code': 'en', 'result_type': 're...",NaN,"{'id': 1200660601057562624, 'id_str': '1200660...",Flybe,-0.5562


In [8]:
#Rekursiv funktion som bygger upp ett träd med tweet-id:s baserat på tweet-trådar, konversationer, där föräldrarna är initial-tweetsens id, och barnen är svarens id:s.  
def build_threads(all_tweets, initial_tweets):
    subthreads = dict.fromkeys(initial_tweets.index, None) #Skapar ny dictionary med key: id och value: None
    for index in initial_tweets.index:
        if index in subthreads.keys():
            responses = all_tweets[all_tweets.in_reply_to_status_id == index]
            if not responses.empty:
                subthreads[index] = build_thread(tweets, responses)
    return subthreads

In [9]:
#Anropar rekursiva funktionen ovan
t_start = time.time()

tweets = data.filter(items=["in_reply_to_status_id"])
initial_tweets = tweets[tweets.in_reply_to_status_id.isnull()]
threads = build_threads(tweets, initial_tweets)

print("Time - build conversation threads:", time.time()-t_start)

149.90031623840332


In [16]:
#Skapar en pretty printer för att kunna printa dictionaries, och printar de 100 första twittertrådarna 
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(dict(list(threads.items())[0:100])) 

{   1238824448544948226: None,
    1238863434533527552: None,
    1238879275631226884: {   1238884226914754561: {1238885614998761505: None},
                             1238884746329030657: None,
                             1238884932216389640: {   1238886079140429825: {   1238886373660164098: {   1238891392635731968: {   1238893271197286409: None}}}}},
    1238879476857155584: None,
    1238899135409016833: {1239599823873081345: None},
    1238906624695205888: None,
    1238907070881107969: None,
    1238926947608080386: None,
    1238934990601302017: None,
    1238944454804996096: None,
    1238961950161240066: None,
    1238963632710582275: {   1238963870678552576: None,
                             1239575150141546496: None,
                             1239648808310648832: None},
    1238967065383444480: None,
    1238969081006563328: None,
    1238999014277775366: None,
    1239009249663090688: None,
    1239040290947809280: None,
    1239105083071254529: None,
    123910749139

In [ ]:
#Funktion för att rensa tweetsen på onödiga symboler
def clean(tweet_str):
    tweet_str = re.sub(r'RT[\s]+', '', tweet_str)
    tweet_str = re.sub(r'https?:\/\/.*[\r\n]*','',tweet_str)
    tweet_str = re.sub(r'#', '', tweet_str)
    return tweet_str

In [ ]:
#Funktion som analyserar och returnerar sentimentet för en tweet
def analyse_sentiment(tweet_str):
    clean_tweet = clean(tweet_str)
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(clean_tweet)
    return sentiment["compound"]

In [ ]:
#Går igenom alla tweets och anropar funktion för att anlysera dess sentiment, vilket läggs till som en ny kolumn i dataframen data. OBS: Tar LÅNG tid, 35-45 min för ~180000 tweets. 
t_start = time.time()
data["sentiment"] = data.full_text.apply(lambda x: analyse_sentiment(x))
print("Time - analysera sentiment:", time.time()-t_start)

In [ ]:
#Uppdatera csv filen
data.to_csv('data.csv')